In [1]:
from bs4 import BeautifulSoup
from fnmatch import fnmatch
from PIL import Image
from PIL import ImageFile
from pathlib import Path
from shutil import copyfile

import base64
import imagehash
import mysql.connector
import os
import re

In [2]:
# GENERAL CONFIGS

ImageFile.LOAD_TRUNCATED_IMAGES = True

# == Database config ==
conn = mysql.connector.connect(
  host="localhost",
  user="root",
  password="root"
)

cursor = conn.cursor()

DB_NAME = "blp_dataset"
TABLE_NAME = "products_tmp_white_rabbit"


# == Files config ==
MAIN_FOLDER = r"D:\dnmarchives\novo\whiterabbit"

In [3]:
# Creating environment
cursor.execute("CREATE DATABASE IF NOT EXISTS {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME))
cursor.execute("USE {}".format(DB_NAME))

In [4]:
table_schema = (
    "CREATE TABLE IF NOT EXISTS products_tmp_white_rabbit ("
    "  id INT(11) AUTO_INCREMENT PRIMARY KEY NOT NULL,"
    "  name TEXT NOT NULL,"
    "  description TEXT,"
    "  price VARCHAR(50),"
    "  market_name VARCHAR(255),"
    "  seller_name TEXT,"
    "  has_image VARCHAR(25) NOT NULL DEFAULT 'has_image',"
    "  illegal ENUM('t', 'f') NOT NULL DEFAULT 't'"
    ") ENGINE=InnoDB")

cursor.execute(table_schema)

In [5]:
def update_image(product_id, value):
    query = "UPDATE products_tmp_white_rabbit SET has_image = %s WHERE id = %s"
    cursor.execute(query, (value, product_id))
    conn.commit()

In [6]:
def get_content(content):
    clear_content = ""

    if content is not None:
        raw_c = content.text

        clear_content = re.sub(r'[\t\r\n]', ' ', raw_c)
        clear_content = re.sub(' +',' ', clear_content).lstrip()
        
    return clear_content

In [8]:
folders = [x for x in Path(MAIN_FOLDER).iterdir() if x.is_dir()]

for fo in folders:
    folder = Path(f"{fo}\items\item")
    folder_image = Path(f"{fo}")

    if folder.exists():
        for file in os.listdir(folder):
            page_content = BeautifulSoup(open(Path(f"{folder}\{file}"), encoding='utf8').read())

            if page_content is not None:
                ## GET DESCRIPTION
                description = ""
                raw_desc = page_content.find("div", attrs={"id": "description"})

                if raw_desc is not None:
                    description = get_content(raw_desc)
            
                ## GET TITLE
                title = ""
                raw_title = page_content.find("h2")
                
                if raw_title is not None:
                    title = get_content(raw_title)
                
                ## GET SELLER
                seller = ""
                raw_seller = page_content.find("table", attrs={"class": "table"})

                if raw_seller is not None:
                    raw_seller = raw_seller.find("a")
                    
                    if raw_seller is not None:
                        seller = get_content(raw_seller)

                if description != "" or title != "":
                    # Insert product into database
                    query = "INSERT INTO products_tmp_white_rabbit (name, description, market_name, seller_name, illegal) VALUES (%s, %s, %s, %s, %s)"
                    values = (title, description, "White Rabbit", seller, "t")
                    
                    cursor.execute(query, values)
                    conn.commit()

                    product_id = cursor.lastrowid

                    ## GET IMAGE
                    img_content = page_content.find("img", attrs={"class": "item-view-img"})
                    
                    if img_content is not None:
                        image = img_content["src"]

                        try:
                            data = "" + image.replace("data:image/jpeg;base64,", "")
                            imgdata = base64.b64decode(data)

                            img_filename = Path(f"D:\images\white_rabbit\{product_id}.jpg")

                            with open(img_filename, 'wb') as f:
                                f.write(imgdata)
                                f.close
                        except:
                            print(f"Image {product_id} has error. Except")
                            update_image(product_id, "no_image")
                    else:
                        print(f"Product {product_id} has no image")
                        update_image(product_id, "no_image")

Image 10 has error. Except
Image 26 has error. Except
Image 34 has error. Except
Image 43 has error. Except
Image 51 has error. Except
Image 60 has error. Except
Image 68 has error. Except
Image 77 has error. Except
Image 120 has error. Except
Image 122 has error. Except
Image 164 has error. Except
Image 172 has error. Except
Image 181 has error. Except
Image 221 has error. Except
Image 308 has error. Except
Image 336 has error. Except
Image 345 has error. Except
Image 385 has error. Except
Image 472 has error. Except
Image 508 has error. Except
Image 547 has error. Except
Image 572 has error. Except
Image 612 has error. Except
Image 694 has error. Except
Image 729 has error. Except
Image 752 has error. Except
Image 784 has error. Except
Image 820 has error. Except
Image 899 has error. Except
Image 934 has error. Except
Image 953 has error. Except
Image 982 has error. Except
Image 1028 has error. Except
Image 1064 has error. Except
Image 1142 has error. Except
Image 1177 has error. Exc